In [22]:
# Insallation des librairies
!pip install requests
!pip install beautifulsoup4
!pip install transformers faiss-cpu sentence-transformers
!pip install gradio



In [30]:
# Importation des librairies
import requests # scraper l'article
from bs4 import BeautifulSoup # scraper l'article
import faiss
import numpy as np
import pandas as pd
import re
from transformers import pipeline# Modèle RAG
from gradio import Interface # Interface web
from sentence_transformers import SentenceTransformer # Modèle RAG



In [24]:
# URL du site web
url = 'https://www.agenceecofin.com/gestion-publique/0109-121155-le-niger-et-le-nigeria-annoncent-un-front-commun-contre-le-terrorisme'

# Obtenir le contenu de la page
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')



In [25]:
# On cherche l'élément contenant le texte de l'article
article = soup.find_all('p', class_="texte textearticle")

In [26]:
# Extraire le texte de l'article
if article: # Check to make sure article is not None
    article_text = ' '.join([p.text.strip() for p in article]) # Extract the text from each paragraph and join them
    print(article_text)
else:
    print("Article not found.")

(Agence Ecofin) - Cette collaboration répondrait à la volonté du Niger de réadapter ses partenariats stratégiques dans la lutte contre le fléau terroriste, alors que les troupes américaines, françaises et allemandes basées sur son territoire dans ce cadre sont en train de se retirer. En visite à Niamey au Niger mercredi, le général Christopher Gwabin Musa, chef d'état-major de la défense du Nigeria a annoncé que les 2 pays vont s’allier pour relever les défis sécuritaires qu’ils rencontrent, notamment en matière de lutte anti-terroriste. « Ce qui s'est passé est passé, nous avons compris qu'il y a des gens qui veulent nous opposer et qui trouvent leur compte dans notre désunion. Nous ne les suivrons pas, car notre devoir est d'assurer la sécurité de nos 2 pays » a-t-il déclaré à la presse, faisant référence aux tensions générées avec la junte nigérienne lorsqu’il annonçait il y a quelques mois une intervention militaire de la CEDEAO pour rétablir l’ordre constitutionnel et le président

In [27]:
# l'enregistrer dans un fichier:
with open('article.txt', 'w', encoding='utf-8') as f:
    f.write(article_text)

In [28]:
data=[article_text]


In [29]:
# Préparer l'article
sentences = article_text.split('.') # Access the string within the list

In [31]:
# Charger les modèles
model_name = "all-MiniLM-L6-v2"
model = SentenceTransformer(model_name)
generator = pipeline("text-generation", model="gpt2")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [32]:
# Encoder les phrases et créer l'index
sentence_embeddings = model.encode(sentences)
dimension = sentence_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(sentence_embeddings)

In [33]:
# Fonction pour générer une réponse
def generate_response(question):
    # Encoder la question
    query_embedding = model.encode(question)

    # Rechercher les k plus proches voisins
    k = 5  # Nombre de phrases à retourner
    distances, indices = index.search(query_embedding, k)

    # Récupérer les phrases les plus similaires
    most_similar_sentences = [sentences[i] for i in indices[0]]

    # Construire le prompt
    prompt = f"Question: {question}\nContexte: {'. '.join(most_similar_sentences)}\nRéponse:"

    # Générer la réponse
    generated_text = generator(prompt=prompt, max_length=100)[0]['generated_text']

    return generated_text

In [34]:
# Interface Gradio
demo = Interface(
    fn=generate_response,
    inputs=["text"],  # Question et article
    outputs="text"
)
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://cbaeaf46f4f2f5eef9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
